In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.svm import SVR
from sklearn.decomposition import PCA

#Colab use
#from google.colab import files
#uploaded = files.upload()
#import io

In [2]:
#standardization
def standard_scaler(X_train, X_test):
    
    scaler = StandardScaler()
    
    X_train = scaler.fit_transform(X_train) #standardize based on train data
    X_test = scaler.transform(X_test) #standardize test data
    
    return X_train, X_test

In [3]:
def evaluate(y_true, y_pred, label='test'):
    mse = mean_squared_error(y_true, y_pred) #calculate MSE
    rmse = np.sqrt(mse) #calculate RMSE
    variance = r2_score(y_true, y_pred) #calculate R2
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

# Load Data

In [4]:
index_names = ['ID', 'Cycle']
setting_names = ['OpSet1', 'OpSet2', 'OpSet3']
sensor_names = ['SensorMeasure{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names

train_data = pd.read_csv("../Data/train_set.csv")
test_data = pd.read_csv("../Data/test_set.csv")
true_RUL = pd.read_csv("../Data/RUL_FD001.txt", sep='\s+', header = None)

#Colab use
#train_data = pd.read_csv(io.BytesIO(uploaded['train_set.csv']))
#test_data = pd.read_csv(io.BytesIO(uploaded['test_set.csv']))
#true_RUL = pd.read_csv(io.BytesIO(uploaded['RUL_FD001.txt']), sep='\s+', header = None)

train_RUL = train_data['RUL']
train_RUL = train_RUL.clip(upper = 125) #clip maximum cycle at 125
test_RUL = test_data['RUL']

train_data = train_data.drop(['RUL'], 1)
test_data = test_data.drop(['RUL'], 1)

test_data = test_data.groupby(['ID'])
test_data = test_data.tail(1)

#assign to new variable for easy understanding
train = train_data
train_y = train_RUL
test = test_data.groupby(['ID']).tail(1) #get the last record for each engine
test_y = true_RUL

#only sensor value considered
train = train[sensor_names]
test = test[sensor_names]

# Feature Extraction

In [5]:
#Principle Component Analysis
def pca(X_train, X_test, y_train, y_test):
    
    #for reproducible result
    np.random.seed(2)
    
    X_train, X_test = standard_scaler(X_train, X_test) #standardize data
    
    #rename columns after standardization
    sensor_names = ['SensorMeasure{}'.format(i) for i in range(1,22)] 
    X_train = pd.DataFrame(X_train, columns = sensor_names).reset_index(drop = True)
    X_test = pd.DataFrame(X_test, columns = sensor_names).reset_index(drop = True)

    #-------------------------------Fitting Model----------------------------
    # Make an instance of the Model
    pca = PCA(0.95, random_state = 1) #95% of the variance (information) is retained

    x1 = pca.fit_transform(X_train) #PCA based on train data
    x2 = pca.transform(X_test) #transform test data
    
    #Graph to indicate information contains in each component
    # fir = plt.figure(figsize=(8,5))
    # sing_vals = np.arange(len(pca.components_)) + 1
    # plt.plot(sing_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
    # plt.title('Scree Plot', fontsize = 20)
    # plt.xlabel('Principal Component', fontsize = 20)
    # plt.ylabel('Eigenvalue', fontsize = 20)
    # plt.xticks(fontsize=10)
    # plt.yticks(fontsize=10)
    #------------------------------------------------------------------------
    
    #convert to data frame
    X_train = pd.DataFrame(data = x1)
    X_test = pd.DataFrame(data = x2)
    
    return X_train, X_test, y_train, y_test

# Support Vector Regression

In [6]:
def support_vector_regression(X_train, X_test, y_train, y_test):
    
    #for reproducible result
    np.random.seed(2)
    
    #------------------------------Train Model-------------------------------
    SVRM = SVR()
    
    c = [int(x) for x in np.linspace(1, 15, num = 15)] #different value for C
    epsilon = [int(x) for x in np.linspace(1, 20, num = 20)] #different value for epsilon

    #Create the dictionary
    param_grid = {'C': c,
                  'epsilon': epsilon}
    
    #K-fold cross validation
    cv = KFold(n_splits=10, shuffle = True, random_state = 1)
    
    #randomized search 
    rand_search = RandomizedSearchCV(SVRM, param_distributions = param_grid, 
                                     cv = cv, n_jobs=-1, random_state=1)
    
    rand_search.fit(X_train, y_train.values.ravel()) #train model
    
    #SVRM.fit(X_train, y_train.values.ravel()) #train model on default settings
    #------------------------------------------------------------------------

    #------------------------------Predict X---------------------------------
    y_pred_train = rand_search.predict(X_train) #predict on train data
    y_pred_test = rand_search.predict(X_test) #predict on test data
    
    #for default model
    #y_pred_train = SVRM.predict(X_train)
    #y_pred_test = SVRM.predict(X_test)
    #------------------------------------------------------------------------
    
    #---------------------------------Accuracy-------------------------------
    # Use score method to get accuracy of model
    accuracy_score = rand_search.score(X_test, y_test)
    #accuracy_score = SVRM.score(X_test, y_test) #for default model
    print('Accuracy of Support Vector Regression on test set: {:.2f}'.format(accuracy_score))
    #------------------------------------------------------------------------
    
    #---------------------------------RMSE & R2------------------------------
    evaluate(y_train, y_pred_train, 'Train')
    evaluate(y_test, y_pred_test, 'Test')
    #------------------------------------------------------------------------
    
    #---------------------------------Best Param-----------------------------
    print(rand_search.best_params_) #parameters that provide best result
    #------------------------------------------------------------------------
    
    filename = '../Dashboard UI/Dashboard UI/model/SVR_model.sav'
    pickle.dump(rand_search, open(filename, 'wb')) #save support vector regression model
    
    #Colab use
    #filename = 'SVR_model.sav'
    #pickle.dump(rand_search, open(filename, 'wb')) #save support vector regression model
    #files.download('SVR_model.sav')

# Working

In [7]:
X_train, X_test, y_train, y_test = pca(train, test, train_y, test_y)

support_vector_regression(X_train, X_test, y_train, y_test)

Accuracy of Support Vector Regression on test set: 0.80
Train set RMSE:18.134825968148295, R2:0.810624524045426
Test set RMSE:18.37509717228015, R2:0.8044761480017149
{'epsilon': 10, 'C': 10}
